# Get data from and save snapshot in S3

In [1]:
import pandas as pd
import boto3
from tqdm.notebook import trange, tqdm

from utils import MODEL_NAME
from utils.s3 import GLOBAL_DATA_BUCKET, MODELS_BUCKET, read_csv_from_s3

In [2]:
import os
os.getcwd()

'c:\\Users\\Anton\\Desktop\\s2\\attack-detection-v1'

In [3]:
TABLE_PREFIX = 'V3'#"v1"

In [4]:
validated_cases = pd.read_csv("V3_.csv")#pd.read_csv("Validated_V0_Desriptions.csv")

In [5]:
validated_cases_old = pd.read_csv("Validated_V0_Desriptions.csv")

In [6]:
validated_cases_old

,File Names,Attack Link,Exploit Types,Minimum Time,train/test splits,Number of Labeled Transactions,Number of Transactions,Validated,Sum of Stolen Amount
0,Bo_Shen001,https://defiyield.app/rekt-database/bo_shen,Access Control,30/11/2016 12:41,Train,2,144,Yes,4.013972e+07
1,Bancor001,https://www.apriorit.com/dev-blog/554-bancor-e...,Access Control,10/06/2017 14:27,Train,2,580,Yes,3.126104e+06
2,Stazie001,https://coinlive.me/how-did-the-founder-of-nft...,Access Control,26/01/2018 20:36,Train,3,1710,Yes,1.999709e+06
3,Taylor001,https://www.zdnet.com/article/all-of-cryptocur...,Access Control,23/03/2018 15:59,Train,1,7,Yes,1.715527e+06
4,MerkleTreeVentures001,NaN,Access Control,10/05/2018 15:25,Train,2,21,Yes,8.460593e+04
...,...,...,...,...,...,...,...,...,...
156,FakeHeroTokenSale006,NaN,Phishing,01/06/2017 13:29,Train,1,505,Yes,4.391695e+03
157,FakeAVA Token Sale001,NaN,Phishing,31/05/2017 17:22,Train,1,118,Yes,2.389645e+03
158,FakeHeroTokenSale005,NaN,Phishing,29/05/2017 21:59,Train,1,100,Yes,1.594864e+04
159,Pando001,https://defiyield.app/rekt-database/pando,Access Control,05/11/2022 11:54,Test,11,102,Yes,1.767291e+07


In [7]:
validated_cases

,File Names,Sender ID,Exploit Types,train/test splits,Minimum Date
0,0xhabitat001,0xc97f82c80df57c34e84491c0eda050ba924d7429,Phishing,Train,3/17/2021 12:11
1,Abracadabra001,0xf5bce5077908a1b7370b9ae04adc565ebd643966,Price Manupilation,Test,11/30/2021 11:47
2,Agora001,0xdef1c0ded9bec7f1a1670819833240f027b25eff,Flash Loan Attack,Train,04/06/2022 04:17
3,Agora002,0x2796317b0ff8538f253012862c06787adfb8ceb6,Flash Loan Attack,Train,03/01/2022 21:58
4,Akropolis001,0x7967ada2a32a633d5c055e2e075a83023b632b4e,Flash Loan Attack,Train,8/21/2020 11:23
...,...,...,...,...,...
159,Zenith002,0xe7ccf947a4ca18998048995c5428245f031bd5c3,Rug pull,Train,10/18/2020 0:54
160,Zenith003,0xb03501ab31e942c7e1af435bc56f4220fea9a06d,Rug pull,Train,02/03/2018 18:13
161,Zenith004,0xd1fc8e7d48974880fcdfc89ca4bc1e115b6fa577,Rug pull,Train,11/14/2020 22:06
162,Zenith005,0xb22dfb715a7b454d83b2fb95e4d38aaf5f3c84fa,Rug pull,Train,05/07/2020 18:36


## Get data from s3

In [8]:
data_bucket = boto3.resource("s3").Bucket(GLOBAL_DATA_BUCKET)

files = TABLE_PREFIX + "/" + validated_cases["File Names"] + ".csv"

data = []

t = tqdm(files)
for f in t:
    t.set_description(f"loading {f}")
    df = read_csv_from_s3(data_bucket, f, {"parse_dates": ["Block_Time"]})
    df["file_name"] = f
    data.append(df)

data = pd.concat(data, axis=0)

data.shape

  0%|          | 0/164 [00:00<?, ?it/s]

(951863, 22)

## Initial cleanup of column names and datetime data

In [9]:
data.columns = (
    data.columns.str.replace("[\[\]]", "", regex=True)
    .str.replace(" ", "_", regex=False)
    .str.lower()
)

In [10]:
data["block_time"] = data["block_time"].dt.tz_localize(None)

In [11]:
data.columns

Index(['transaction_id', 'transaction_time', 'block_time', 'sender_id',
       'receiver_id', 'receiver_type', 'amount', 'currency', 'currency_hash',
       'currency_type', 'amount_usd', 'gas_price', 'gas_limit', 'gas_used',
       'gas_predicted', 'balance_in', 'balance_out', 'label', 'risk_level',
       'file_name', 'unnamed:_15', 'unnamed:_16'],
      dtype='object')

In [12]:
data.drop(["unnamed:_15", "unnamed:_16"], axis=1, inplace=True)

## Save dataset locally and backup to s3

In [13]:
data.to_csv("data/raw_data.csv", index=False)

In [ ]:
models_bucket = boto3.resource("s3").Bucket(MODELS_BUCKET)

In [ ]:
models_bucket.upload_file("data/raw_data.csv", f"{MODEL_NAME}/data/raw_data.csv")